# Labo 6 : désambiguïsation lexicale (Word Sense Disambiguation)
Par Vincent Guidoux et Nathan Gonzalez Montes

## Objectif et informations

Comparer deux méthodes de désambiguïsation du sens des mots en contexte (tâche notée WSD), l’une utilisant l’algorithme de Lesk simplifié, et l’autre utilisant word2vec.

Le labo utilisera une ressource contenant 2'369 occurrences du mot anglais « interest » annotées chacune avec le sens du mot dans le contexte respectif. Chacune des deux méthodes fonctionne selon le même principe général : comparer le contexte de l’occurrence avec les définitions des sens, et choisir la définition la plus proche. L’algorithme de Lesk définit la proximité comme le nombre de mots en commun, alors que word2vec peut la calculer comme la similarité de deux vecteurs (somme des vecteurs de mots).

Dans une première approche, qui ne nécessite pas de programmation, on vous demande de comparer une occurrence de chaque sens. Dans une seconde approche, qui nécessite un peu de programmation en Python, vous évaluerez les deux systèmes sur l’ensemble des 2'369 occurrences.

## Étapes

### 1. Le fichier de données se trouve à http://www.d.umn.edu/~tpederse/data.html – chercher « interest » vers la fin de la page, et prendre le fichier marqué « original format without POS tags ». Quel est le format de ce fichier et comment sont annotés les sens ?

Le fichier est au format texte.

Le sens **1** d'_interest_ sera noté `interest_1`, le sens **2** d'_interest_ sera noté `interest_2`, etc..

### 2. Bien lire le fichier README associé. Quelles sont les définitions des six sens de « interest » annotées dans les données ? De quel dictionnaire viennent-elles ? Où se trouve-t-il en ligne ?

1. Readiness to give attention
2. Quality of causing attention to be given to
3. Activity, etc. that one gives attention to
4. Advantage, advancement or favor
5. A share in a company or business
6. Money paid for the use of money

Du dictionnaire LDOCE. Ici: [Longman Dictionary of Contemporary English | LDOCE
](https://www.ldoceonline.com/)


### 3. Consulter WordNet en ligne et identifier les définitions correspondant aux six sens annotés dans les données. En les combinant éventuellement avec les résultats du (2), écrivez une liste de mots « pleins » (i.e. sans stopwords) pour chaque définition.

### Noun

1. A sense of concern with and curiosity about someone or something, **involvement**: *"an interest in music"*

2. The power of attracting or holding one's attention (because it is unusual or exciting etc.), **interestingness**: *"they said nothing of great interest"; "primary colors can add interest to a room"e or something) "an interest in music"*

3. A diversion that occupies one's time and thoughts (usually pleasantly), **pastime, pursuit**: *"sailing is her favorite pastime"; "his main pastime is gambling"; "he counts reading among his interests"; "they criticized the boy for his limited pursuits"*

4. A reason for wanting something done, **sake**: *"for your sake"; "died for the sake of his country"; "in the interest of safety"; "in the common interest"*

5. (law) A right or legal share of something; a financial involvement with something, **stake**: *"they have interests all over the world"; "a stake in the company's future"*

6. A fixed charge for borrowing money; usually a percentage of the amount borrowed: *"how much interest do you pay on your mortgage?"*

(usually plural) a social group whose members control some field of activity and who have common aims, interest group: *"the iron interests stepped up production"*

### Verb

Excite the curiosity of; engage the interest of

**Concern, occupy, worry**, be on the mind of: "I worry about the second Germanic consonant shift"

**Matter to**, be of importance or consequence: "This matters to me!"

1. Readiness, give, attention, sense, concern, curiosity, someone, something, involvement
2. Qualitiy, causing, attraction, be, given, power, attracting, holding, one's, attention, because, unusual, exciting
3. Activity, one, gives, attention, diversion, occupies, one's, time, thoughts, usually, pleasantly, pastime, pursuit

Liste de mots plein :

1. "readiness, give, attention, sense, concern, curiosity, someone, something, involvement"
2. "qualitiy, causing, attraction, be, given, power, attracting, holding, one, attention, because, unusual, exciting"
3. "activity, one, gives, attention, diversion, occupies, one, time, thought, usually, pleasantly, pastime, pursuit"
4. "reason wanting something do sake die sake country interest safety common interest advantage advancement favor"
5. "law right legal share something financial involvement something stake they have interest world stake company future share company business"
6. "fixed charge for borrowing money usually percentage amount borrowed interest pay mortgage social group members control field activity have common aims interest group iron interests stepped up production money paid use money"

### 4. Considérez la première occurrence de « interest » dans les données, et notez les mots « pleins » qui entourent « interest » (p.ex. toute la phrase). Combien de mots y’a-t-il en commun avec chacune des six définitions ? Quel est donc le sens le plus probable ?

In [9]:
first_occurance = "yield, money, market, mutual, fund, continue, slide, amid, sign, portfolio, manager, expect, further, decline, rates"
first_occurance = first_occurance.split(", ")

definition_1 = "readiness, give, attention, sense, concern, curiosity, someone, something, involvement"
definition_1 = definition_1.split(", ")

definition_2 = "quality, causing, attraction, be, given, power, attracting, holding, one, attention, because, unusual, exciting"
definition_2 = definition_2.split(", ")

definition_3 = "activity, one, gives, attention, diversion, occupies, one, time, thought, usually, pleasantly, pastime, pursuit"
definition_3 = definition_3.split(", ")

definition_4 = "reason wanting something do sake die sake country interest safety common interest advantage advancement favor"
definition_4 = definition_4.split(" ")

definition_5 = "law right legal share something financial involvement something stake they have interest world stake company future share company business"
definition_5 = definition_5.split(" ")

definition_6 = "fixed charge for borrowing money usually percentage amount borrowed interest pay mortgage social group members control field activity have common aims interest group iron interests stepped up production money paid use money"
definition_6 = definition_6.split(" ")

In [10]:
def scoring(list_input, list_does_contains):
    score = 0

    for word in list_input:
        if word in list_does_contains:
            score = score + 1
    return score    

In [11]:
print("Score comparaison avec la définition 1 : ", scoring(first_occurance, definition_1))
print("Score comparaison avec la définition 2 : ", scoring(first_occurance, definition_2))
print("Score comparaison avec la définition 3 : ", scoring(first_occurance, definition_3))
print("Score comparaison avec la définition 4 : ", scoring(first_occurance, definition_4))
print("Score comparaison avec la définition 5 : ", scoring(first_occurance, definition_5))
print("Score comparaison avec la définition 6 : ", scoring(first_occurance, definition_6))

Score comparaison avec la définition 1 :  0
Score comparaison avec la définition 2 :  0
Score comparaison avec la définition 3 :  0
Score comparaison avec la définition 4 :  0
Score comparaison avec la définition 5 :  0
Score comparaison avec la définition 6 :  1


Il n'y a qu'un mot commun, et c'est avec la définition bonne définition, nous avons eu de la chance

### 5. En réutilisant le modèle de word2vec entraîné sur Google News fourni par Gensim, calculer les similarités (cosinus) entre les mots du contexte de la première occurrence (somme(ou moyenne) des vecteurs de chaque mot) et chacune des six définitions. Quel est donc le sens le plus probable ?

**Suggestion** : chercher dans la documentation word2vec comment obtenir très facilement la similarité entre deux ensembles de mots.

In [12]:
import numpy
import nltk
import time
from gensim.test.utils import common_texts
from gensim.models import Word2Vec
from gensim.test.utils import get_tmpfile
from gensim.models import KeyedVectors
from gensim.test.utils import datapath

w2v_vectors = KeyedVectors.load_word2vec_format("word2vec-google-news-300.gz", binary=True)

In [14]:
definitions = [definition_1, definition_2, definition_3, definition_4, definition_5, definition_6]

i = 1
for definition in definitions:
    print("Similarité entre la définition {} et la première occurance de interest : {}".format(i,  w2v_vectors.n_similarity(first_occurance, definition)))
    i += 1

Similarité entre la définition 1 et la première occurance de interest : 0.360268771648407
Similarité entre la définition 2 et la première occurance de interest : 0.43276697397232056
Similarité entre la définition 3 et la première occurance de interest : 0.32915887236595154
Similarité entre la définition 4 et la première occurance de interest : 0.4701043963432312
Similarité entre la définition 5 et la première occurance de interest : 0.5618512630462646
Similarité entre la définition 6 et la première occurance de interest : 0.6260374784469604


Le sens le plus probable est le 6 ! Youpi, c'est ce que nous voulions !

### 6. Approche manuelle : appliquer la procédure précédente à une occurrence de « interest » pour chaque sens possible (six sens, donc six mots). Combien sur les six sont correctement désambiguïsés par l’algorithme de Lesk, et combien par la méthode word2vec ?

In [15]:
interest_1 = "the purchases  show  the strong interest_1  of  japanese investors  in  US mortgage-based instruments  ,  fannie mae 's chairman  ,  david maxwell  , said at  a news conference"

interest_2 = "abc 's baseball experience  may be of  interest_2  to  cbs inc.  ,  which   next season  takes over  the broadcasting  of  all baseball playoffs  in  a four-year television contract  priced at  $ billion  ."

interest_3 = "blue-chip advertisers  have  plenty  of  complaints  about  the magazines they  advertise in , ranging from  inadequate consumer research  to  ad  clutter and  a  seemingly  unchecked proliferation  of  special interest_3 magazines  ."

interest_4 = "insofar as  mister  has  a special interest_4  in  nih  and  the institutions   that  receive  its  funding ,  the rockefeller scientists  were  no doubt  discomfited by  doctor baltimore 's unflattering public opinion  of  this congressional patron  ,  whose behavior  reminded  doctor baltimore  of  the mccarthy era  ."

interest_5 = "the spokesman  refused to comment on  speculation  that  industrial equity  might use  its interest_5  as  a platform  to launch  a hostile bid  for  the company  ."

interest_6 = "indeed ,  the banker  says , in  the reborn capital structure cash coverage  of  interest_6  will meaningfully improve ."

interests = [interest_1, interest_2, interest_3, interest_4, interest_5, interest_6]

In [16]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
nltk.download('stopwords')

def remove_stop_words(data):
    stopWords = set(stopwords.words('english'))
    stopWords.add(',')
    stopWords.add('.')
    stopWords.add('\'s')
    stopWords.add(' ')
    stopWords.add('')
    stopWords.add('interest_1')
    stopWords.add('interest_2')
    stopWords.add('interest_3')
    stopWords.add('interest_4')
    stopWords.add('interest_5')
    stopWords.add('interest_6')
    words = word_tokenize(data)
    wordsFiltered = []

    for w in words:
        if w not in stopWords:
            wordsFiltered.append(w)
    
    return wordsFiltered

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Nathan\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [17]:
definitions = [definition_1, definition_2, definition_3, definition_4, definition_5, definition_6]

i = 1
for interest in interests:
    print("\n----------------\n")
    
    interest = interest.replace('-', ' ')
    interest = remove_stop_words(interest)
    wv_biggest_similarity = 0
    wv_bestest_definition = None
    
    lesk_biggest_similarity = 0
    lesk_bestest_definition = None
    
    j = 1
    for definition in definitions:
        wv_current_similarity = w2v_vectors.n_similarity(interest, definition)
        lesk_current_similarity = scoring(interest, definition)
        
        if lesk_biggest_similarity < lesk_current_similarity:
            lesk_biggest_similarity = lesk_current_similarity
            lesk_bestest_definition = j
        
        if wv_biggest_similarity < wv_current_similarity:
            wv_biggest_similarity = wv_current_similarity
            wv_bestest_definition = j
        j += 1
    print("word2vec : La meilleur similarité({}) est entre la définition {} et la première occurance de interest_{}".format(wv_biggest_similarity, wv_bestest_definition, i))
    print("Lesk : La meilleur similarité({}) est entre la définition {} et la première occurance de interest_{}".format(lesk_biggest_similarity, lesk_bestest_definition, i))
    i += 1


----------------

word2vec : La meilleur similarité(0.48168566823005676) est entre la définition 6 et la première occurance de interest_1
Lesk : La meilleur similarité(1) est entre la définition 6 et la première occurance de interest_1

----------------

word2vec : La meilleur similarité(0.5006052255630493) est entre la définition 6 et la première occurance de interest_2
Lesk : La meilleur similarité(0) est entre la définition None et la première occurance de interest_2

----------------

word2vec : La meilleur similarité(0.4979081153869629) est entre la définition 6 et la première occurance de interest_3
Lesk : La meilleur similarité(0) est entre la définition None et la première occurance de interest_3

----------------

word2vec : La meilleur similarité(0.5013031363487244) est entre la définition 1 et la première occurance de interest_4
Lesk : La meilleur similarité(0) est entre la définition None et la première occurance de interest_4

----------------

word2vec : La meilleur simi

### 7. Approche automatique : implémenter un court programme qui applique la méthode décrite ci-dessus à chacune des 2'369 occurrences de « interest » dans le corpus du test.

Il faut donc : extraire les mots voisins de chaque occurrence, compter le nombre de mots en commun avec chaque définition, et choisir le sens qui maximise ce nombre (Lesk) ; également, calculer la similarité word2vec entre le contexte et chaque définition, et choisir le sens qui la maximise.

In [9]:
filepath = 'data/interest-original.txt' 

try:  
    fp = open(filepath, 'r',encoding="utf-8")
    raw_sentences = fp.read().replace('=', '')
    raw_sentences = raw_sentences.split('\n$$\n')
finally:  
    fp.close()
    
for sentence in raw_sentences:
    sentence = sentence.replace('-', ' ')
    sentence = remove_stop_words(sentence)
    
    wv_biggest_similarity = 0
    wv_bestest_definition = None
    
    j = 1
    for definition in definitions:
        wv_current_similarity = w2v_vectors.n_similarity(interest, definition)
        
        if wv_biggest_similarity < wv_current_similarity:
            wv_biggest_similarity = wv_current_similarity
            wv_bestest_definition = j
        j += 1
    i += 1

Quelles sont les proportions respectives de bonnes réponses de Lesk et de word2vec ?
Pouvez-vous les calculer également pour chaque catégorie ?

### 8. Comment se comparent vos scores avec ceux publiés dans l’article de Pedersen (2000) disponible ici : http://www.aclweb.org/anthology/A00-2009 ?

|                         | accuracy | method           | feature set                |
|-------------------------|:--------:|------------------|----------------------------|
| Naive Bayesian Ensemble | 89%      | ensemble of 9    | varying left & right b-o-w |
| Ng & Lee, 1996          | 87%      | nearest neighbor | p-o-s, morph, co-occur     |
|                         |          |                  | collocates, verb-obj       |
| Bruce & Wiebe, 1994     | 78%      | model selection  | p-o-s, morph, co-occur     |
| Pedersen & Bruce, 1997  | 78%      | decision tree    | p-o-s, morph, co-occur     |
|                         | 74%      | naive bayes      |                            |

|                                   | accuracy | method        | feature set                   |
|-----------------------------------|:--------:|---------------|-------------------------------|
| Naive Bayesian Ensemble           | 88%      | ensemble of 9 | varying left & right b-o-w    |
| Towell & Voorhess, 1998           | 87%      | neural net    | local & topical b-o-w, p.-o-s |
| Leacock, Chodor0w, & Miller, 1998 | 84%      | naive baye    | local & topical b-o-w, p-o-s  |
| Leacock, Towell, & Voorhees, 1993 | 76%      | neural net    | 2 sentence b-o-w              |
|                                   | 72%      | content vector|                               |
|                                   | 71%      | naive bayes   |                               |
| Mooney, 1996                      | 72%      | naive bayes   | 2 sentence b-o-w              |
|                                   | 71%      | perceptron    |                               |

Merci d’envoyer votre notebook Jupyter par email au professeur et à l’assistant, avant le vendredi 17 mai à 23h59.